# // TO DO LIST//

1. combine city csv files ('Atlanta.csv' + 'Atlanta 2.csv.')

use website: https://api.meteostat.net/#history with api key=ipeqdKHL

In [76]:
import pickle
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import requests
import lxml
import json
import os
import re
api_key = 'ipeqdKHL'

In [47]:
dirName = 'HistoricalWeatherData'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print('Directory ', dirName, ' Created')
else:
    print('Directory ', dirName, ' already exists')

Directory  HistoricalWeatherData  already exists


In [32]:
def get_stadium_cities_wiki():
    url = 'https://en.wikipedia.org/wiki/List_of_current_National_Football_League_stadiums'
    table = pd.read_html(url)
    df = table[1]
    
    keep_cols = ['Name', 'Capacity', 'Location', 'Surface', 'Roof type', 'Team(s)', 'Opened']
    
    df = df[keep_cols]
    
    df['Capacity'] = df['Capacity'].astype(int)
    df['Surface'] = df['Surface'].replace(to_replace=r'\[\d{1,2}\]', value='', regex=True)
    df['Opened'] = df['Opened'].replace(to_replace=r'\[.*?\]', value='', regex=True)
    
    # need to fix this regex to properly divide the column into 2
#     df['Team(s)'] = df['Team(s)'].str.split(r'(.*[A-Za-z]+)([A-Z].*)', expand=True)
        
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_from_wiki.pkl')

In [33]:
def get_stadium_cities_nonwiki():
    url = 'https://geojango.com/pages/list-of-nfl-teams'
    table = pd.read_html(url)
    
    df = table[0]
    df[['City', 'State']] = df['Arena Location'].str.extract('^(?P<City>.+?),\s(?P<State>.+)', expand=True)
    
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_not_from_wiki.pkl')
    
    
def get_state_abbrevs():
    url2 = 'https://www.softschools.com/social_studies/state_abbreviations/'
    table2 = pd.read_html(url2)
    df2 = table2[2]
    df2['State'] = df2['State'].str.title()
    df2.set_index('State', inplace=True)
    pd.to_pickle(df2, './state_abbreviations.pkl')

In [34]:
get_stadium_cities_nonwiki()
get_state_abbrevs()

In [35]:
stadiums_df = pd.read_pickle('./stadium_dataframe_not_from_wiki.pkl')
stadiums_df

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,City,State
0,Arizona Cardinals,State Farm Stadium,"Glendale, Arizona",63400,2006,Glendale,Arizona
1,Atlanta Falcons,Mercedes-Benz Stadium,"Atlanta, Georgia",71000,2017,Atlanta,Georgia
2,Baltimore Ravens,M&T Bank Stadium,"Baltimore, Maryland",71008,1998,Baltimore,Maryland
3,Buffalo Bills,New Era Field,"Orchard Park, New York",71608,1973,Orchard Park,New York
4,Carolina Panthers,Bank of America Stadium,"Charlotte, North Carolina",75523,1996,Charlotte,North Carolina
5,Chicago Bears,Soldier Field,"Chicago, Illinois",61500,1924,Chicago,Illinois
6,Cincinnati Bengals,Paul Brown Stadium,"Cincinnati, Ohio",65515,2000,Cincinnati,Ohio
7,Cleveland Browns,First Energy Stadium,"Cleveland, Ohio",67895,1999,Cleveland,Ohio
8,Dallas Cowboys,At&T Stadium,"Arlington, Texas",80000,2009,Arlington,Texas
9,Denver Broncos,Empower Field at Mile High,"Denver, Colorado",76125,2001,Denver,Colorado


In [36]:
state_abbrevs = pd.read_pickle('./state_abbreviations.pkl')
state_abbrevs.to_dict()['Abbreviation']

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY'}

In [37]:
def get_station_id(city, key):
    url = 'https://api.meteostat.net/v1/stations/search?q={}&key={}'.format(city, key)
    response = requests.get(url)
    return response.json()

In [9]:
# test historical weather API
city = 'Baltimore'
station = get_station_id(city, api_key)['data'][0]['id']
station

'72406'

In [48]:
for row in stadiums_df['City']:
    print(row, get_station_id(row, api_key))
    print('\n\n')

Glendale {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72288', 'country': 'US', 'name': 'Burbank-Glendale-Pasadena'}, {'id': 'KGEU0', 'country': 'US', 'name': 'Glendale / Country Greens At Villa De Paz'}, {'id': 'KHVS0', 'country': 'US', 'name': 'Hartsville / Glendale Acres'}, {'id': 'KLUF0', 'country': 'US', 'name': 'Phoenix / Glendale / Webb'}, {'id': 'KORB0', 'country': 'US', 'name': 'Orr / Glendale'}]}



Atlanta {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72219', 'country': 'US', 'name': 'Hartsfield-Jackson Atlanta'}, {'id': 'KCVC0', 'country': 'US', 'name': 'Atlanta'}, {'id': 'KFTY0', 'country': 'US', 'name': 'Atlanta / Carroll Heights'}, {'id': 'KPDK0', 'country': 'US', 'name': 'Atlanta / Northwoods'}]}



Baltimore {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72406', 'country':

Oakland {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72493', 'country': 'US', 'name': 'Metro Oakland International  Airport'}, {'id': 'KWVL0', 'country': 'US', 'name': 'Waterville / Oakland'}, {'id': 'KRJD0', 'country': 'US', 'name': 'Ridgely / Oakland Acres'}]}



Philadelphia {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72408', 'country': 'US', 'name': 'Philadelphia International Airport'}, {'id': 'KPHD0', 'country': 'US', 'name': 'New Philadelphia / Schoenbrunn Estates'}, {'id': 'KPNE0', 'country': 'US', 'name': 'Philadelphia / Ashton Wooden Bridge'}]}



Pittsburgh {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72520', 'country': 'US', 'name': 'Greater Pittsburgh International'}]}



Santa Clara {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdie

In [55]:
def get_historical_data(dirName, city, station_id, key, timezone, start, end):
    url = 'https://api.meteostat.net/v1/history/hourly?station={}&start={}&end={}&time_zone={}&time_format=Y-m-d%20H:i&key={}'
    response = requests.get(url.format(station_id,start,end,timezone,key))
    weather_data = response.json()
    
    with open('{}\{}.json'.format(dirName, city), 'w') as f:
        json.dump(weather_data, f)
    
    print('Successfully wrote to json for ', city, ' from', start, ' to ', end)

In [65]:
get_historical_data(dirName, 'Glendale', 'KLUF0', api_key, 'Europe/London', '2019-01-01', '2019-01-05')

Successfully wrote to json for  Glendale  from 2019-01-01  to  2019-01-05


In [49]:
# need dictionary for cities and their correct station number

cities = {'Glendale':3, 'Atlanta':0, 'Baltimore':1,'Orchard Park':['Buffalo', 0],
          'Charlotte':0, 'Chicago':0,'Cincinnati':0, 'Cleveland':0,
          'Arlington':1, 'Denver':0, 'Detroit':0, 'Green Bay':['Appleton',0],
          'Houston':0, 'Indianapolis':0, 'Jacksonville':0, 'Kansas City':0,
          'Carson':['Hawthorne',0], 'Los Angeles':0, 'Miami Gardens':['Miami',0],
          'Minneapolis':0, 'Foxborough':['Pawtucket',0], 'New Orleans':0,
          'East Rutherford':['Newark', 0], 'Oakland':0, 'Philadelphia':0,
          'Pittsburgh':0, 'Santa Clara':['San Jose',1], 'Seattle':0, 'Tampa':0,
          'Nashville':0, 'Landover':['Washington',0]}

In [50]:
def get_all_station_id(city_dict, key):
    list_of_stations = []
    for k, v in city_dict.items():
        if type(v) == list:
            url = 'https://api.meteostat.net/v1/stations/search?q={}&key={}'.format(v[0], key)
            response = requests.get(url)
            station_json = response.json()['data'][v[1]]
        else:
            url = 'https://api.meteostat.net/v1/stations/search?q={}&key={}'.format(k, key)
            response = requests.get(url)
            station_json = response.json()['data'][v]
        list_of_stations.append({k: station_json})
        
        print('Got station id for ', k)
    
    with open('stations.pkl', 'wb') as f:
        pickle.dump(list_of_stations, f)
        

In [43]:
get_all_station_id(cities, api_key)

Got station id for  Glendale
Got station id for  Atlanta
Got station id for  Baltimore
Got station id for  Orchard Park
Got station id for  Charlotte
Got station id for  Chicago
Got station id for  Cincinnati
Got station id for  Cleveland
Got station id for  Arlington
Got station id for  Denver
Got station id for  Detroit
Got station id for  Green Bay
Got station id for  Houston
Got station id for  Indianapolis
Got station id for  Jacksonville
Got station id for  Kansas City
Got station id for  Carson
Got station id for  Los Angeles
Got station id for  Miami Gardens
Got station id for  Minneapolis
Got station id for  Foxborough
Got station id for  New Orleans
Got station id for  East Rutherford
Got station id for  Oakland
Got station id for  Philadelphia
Got station id for  Pittsburgh
Got station id for  Santa Clara
Got station id for  Seattle
Got station id for  Tampa
Got station id for  Nashville
Got station id for  Landover


In [51]:
with open('stations.pkl', 'rb') as f:
    stations_list = pickle.load(f)

In [52]:
for place in stations_list:
    for k, v in place.items():
        print(k, v)

Glendale {'id': 'KLUF0', 'country': 'US', 'name': 'Phoenix / Glendale / Webb'}
Atlanta {'id': '72219', 'country': 'US', 'name': 'Hartsfield-Jackson Atlanta'}
Baltimore {'id': 'KDMH0', 'country': 'US', 'name': 'Baltimore'}
Orchard Park {'id': '72528', 'country': 'US', 'name': 'Buffalo Airport'}
Charlotte {'id': '72314', 'country': 'US', 'name': 'Charlotte/Douglas International'}
Chicago {'id': '72530', 'country': 'US', 'name': 'Chicago O’hare Airport'}
Cincinnati {'id': '72421', 'country': 'US', 'name': 'Cincinnati/Northern Ky International'}
Cleveland {'id': '72524', 'country': 'US', 'name': 'Cleveland-Hopkins International'}
Arlington {'id': 'KGKY0', 'country': 'US', 'name': 'Arlington / Southland Acres'}
Denver {'id': '72565', 'country': 'US', 'name': 'Denver International Airport'}
Detroit {'id': '72537', 'country': 'US', 'name': 'Detroit Metropolitan'}
Green Bay {'id': 'KAQP0', 'country': 'US', 'name': 'Appleton'}
Houston {'id': '72243', 'country': 'US', 'name': 'Houston Interconti

In [56]:
for place in stations_list:
    for k, v in place.items():
        print(k)
        get_historical_data(dirName, k + ' 2', v['id'], api_key, 'Europe/London', '2010-01-01', '2014-12-31')

Glendale
Successfully wrote to json for  Glendale 2  from 2010-01-01  to  2014-12-31
Atlanta
Successfully wrote to json for  Atlanta 2  from 2010-01-01  to  2014-12-31
Baltimore
Successfully wrote to json for  Baltimore 2  from 2010-01-01  to  2014-12-31
Orchard Park
Successfully wrote to json for  Orchard Park 2  from 2010-01-01  to  2014-12-31
Charlotte
Successfully wrote to json for  Charlotte 2  from 2010-01-01  to  2014-12-31
Chicago
Successfully wrote to json for  Chicago 2  from 2010-01-01  to  2014-12-31
Cincinnati
Successfully wrote to json for  Cincinnati 2  from 2010-01-01  to  2014-12-31
Cleveland
Successfully wrote to json for  Cleveland 2  from 2010-01-01  to  2014-12-31
Arlington
Successfully wrote to json for  Arlington 2  from 2010-01-01  to  2014-12-31
Denver
Successfully wrote to json for  Denver 2  from 2010-01-01  to  2014-12-31
Detroit
Successfully wrote to json for  Detroit 2  from 2010-01-01  to  2014-12-31
Green Bay
Successfully wrote to json for  Green Bay 2  

In [57]:
for place in stations_list:
    for k, v in place.items():
        print(k)
        get_historical_data(dirName, k, v['id'], api_key, 'Europe/London', '2015-01-01', '2019-12-31')

Glendale
Successfully wrote to json for  Glendale  from 2015-01-01  to  2019-12-31
Atlanta
Successfully wrote to json for  Atlanta  from 2015-01-01  to  2019-12-31
Baltimore
Successfully wrote to json for  Baltimore  from 2015-01-01  to  2019-12-31
Orchard Park
Successfully wrote to json for  Orchard Park  from 2015-01-01  to  2019-12-31
Charlotte
Successfully wrote to json for  Charlotte  from 2015-01-01  to  2019-12-31
Chicago
Successfully wrote to json for  Chicago  from 2015-01-01  to  2019-12-31
Cincinnati
Successfully wrote to json for  Cincinnati  from 2015-01-01  to  2019-12-31
Cleveland
Successfully wrote to json for  Cleveland  from 2015-01-01  to  2019-12-31
Arlington
Successfully wrote to json for  Arlington  from 2015-01-01  to  2019-12-31
Denver
Successfully wrote to json for  Denver  from 2015-01-01  to  2019-12-31
Detroit
Successfully wrote to json for  Detroit  from 2015-01-01  to  2019-12-31
Green Bay
Successfully wrote to json for  Green Bay  from 2015-01-01  to  201

In [81]:
# combine json files into singular csv file
weather_path = os.getcwd() + '\\historicalweatherdata'

files = os.listdir(weather_path)

for name in files:
    if '.json' in name:
        print(name)

Arlington 2.json
Arlington.json
Atlanta 2.json
Atlanta.json
Baltimore 2.json
Baltimore.json
Carson 2.json
Carson.json
Charlotte 2.json
Charlotte.json
Chicago 2.json
Chicago.json
Cincinnati 2.json
Cincinnati.json
Cleveland 2.json
Cleveland.json
Denver 2.json
Denver.json
Detroit 2.json
Detroit.json
East Rutherford 2.json
East Rutherford.json
Foxborough 2.json
Foxborough.json
Glendale 2.json
Glendale.json
Green Bay 2.json
Green Bay.json
Houston 2.json
Houston.json
Indianapolis 2.json
Indianapolis.json
Jacksonville 2.json
Jacksonville.json
Kansas City 2.json
Kansas City.json
Landover 2.json
Landover.json
Los Angeles 2.json
Los Angeles.json
Miami Gardens 2.json
Miami Gardens.json
Minneapolis 2.json
Minneapolis.json
Nashville 2.json
Nashville.json
New Orleans 2.json
New Orleans.json
Oakland 2.json
Oakland.json
Orchard Park 2.json
Orchard Park.json
Philadelphia 2.json
Philadelphia.json
Pittsburgh 2.json
Pittsburgh.json
Santa Clara 2.json
Santa Clara.json
Seattle 2.json
Seattle.json
Tampa 2.js

In [74]:
data = json.load(open(weather_path + '\\atlanta.json'))
df = pd.DataFrame(data["data"])
df.head()

,time,time_local,temperature,dewpoint,humidity,precipitation,precipitation_3,precipitation_6,snowdepth,windspeed,peakgust,winddirection,pressure,condition
0,2015-01-01 00:00:00,2015-01-01 00:00,6.1,0.6,68.0,NaN,None,None,None,11.2,None,330.0,1028.6,NaN
1,2015-01-01 01:00:00,2015-01-01 01:00,5.0,0.6,73.0,0.0,None,None,None,9.4,None,330.0,1028.9,NaN
2,2015-01-01 02:00:00,2015-01-01 02:00,4.4,0.0,73.0,0.0,None,None,None,7.6,None,310.0,1029.0,NaN
3,2015-01-01 03:00:00,2015-01-01 03:00,4.4,0.0,73.0,0.0,None,None,None,0.0,None,NaN,1028.5,NaN
4,2015-01-01 04:00:00,2015-01-01 04:00,3.3,-0.5,76.0,0.0,None,None,None,9.4,None,320.0,1028.9,NaN


In [171]:
# regex for matching 'Atlanta.json' or 'Atlanta 2.json' and then can split that to save CSV file r'([^\\]*$)'
renamed_cols = {'time':'Time (GMT)', 'time_local':'Local Time', 'temperature':'Temperature (°C)', 'dewpoint':'Dewpoint (°C)', 'humidity':'Humidity (%)', 'precipitation':'Precipitation (mm)', 
                'windspeed':'Wind Speed (km/h)', 'winddirection':'Wind Direction (deg)', 'pressure':'Air Pressure (hPa)', 'City':'City'}

for name in files:
    if '.json' in name:
        df = json.load(open(weather_path + '\\' + name))
        df = pd.DataFrame(data["data"])
        name = name.split('.')[0]
        if re.match(r'.*[0-9]', name) == None:
            df['City'] = name
        else:
            rename = re.findall(r'(.*)\s[0-9]', name)[0]
            df['City'] = rename
        print('Saved ', name + '.csv')
        df['time'] = pd.to_datetime(df['time'])
        df['time_local'] = pd.to_datetime(df['time_local'])
        df = df.rename(columns=renamed_cols)
        drop_cols = ['Unnamed: 0', 'precipitation_3', 'precipitation_6', 'snowdepth', 'peakgust']
        df = df.drop(columns = [item for item in df.columns.to_list() if item in drop_cols])
        
        df.to_csv(weather_path + '\\' + name + '.csv', header=True, index=False)

Saved  Arlington 2.csv
Saved  Arlington.csv
Saved  Atlanta 2.csv
Saved  Atlanta.csv
Saved  Baltimore 2.csv
Saved  Baltimore.csv
Saved  Carson 2.csv
Saved  Carson.csv
Saved  Charlotte 2.csv
Saved  Charlotte.csv
Saved  Chicago 2.csv
Saved  Chicago.csv
Saved  Cincinnati 2.csv
Saved  Cincinnati.csv
Saved  Cleveland 2.csv
Saved  Cleveland.csv
Saved  Denver 2.csv
Saved  Denver.csv
Saved  Detroit 2.csv
Saved  Detroit.csv
Saved  East Rutherford 2.csv
Saved  East Rutherford.csv
Saved  Foxborough 2.csv
Saved  Foxborough.csv
Saved  Glendale 2.csv
Saved  Glendale.csv
Saved  Green Bay 2.csv
Saved  Green Bay.csv
Saved  Houston 2.csv
Saved  Houston.csv
Saved  Indianapolis 2.csv
Saved  Indianapolis.csv
Saved  Jacksonville 2.csv
Saved  Jacksonville.csv
Saved  Kansas City 2.csv
Saved  Kansas City.csv
Saved  Landover 2.csv
Saved  Landover.csv
Saved  Los Angeles 2.csv
Saved  Los Angeles.csv
Saved  Miami Gardens 2.csv
Saved  Miami Gardens.csv
Saved  Minneapolis 2.csv
Saved  Minneapolis.csv
Saved  Nashville

In [173]:
df = pd.read_csv(weather_path + '\\' + 'glendale.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 11 columns):
Time (GMT)              43824 non-null object
Local Time              43824 non-null object
Temperature (°C)        43823 non-null float64
Dewpoint (°C)           43823 non-null float64
Humidity (%)            43823 non-null float64
Precipitation (mm)      41639 non-null float64
Wind Speed (km/h)       43824 non-null float64
Wind Direction (deg)    42095 non-null float64
Air Pressure (hPa)      43431 non-null float64
condition               14093 non-null float64
City                    43824 non-null object
dtypes: float64(8), object(3)
memory usage: 3.7+ MB


,Time (GMT),Local Time,Temperature (°C),Dewpoint (°C),Humidity (%),Precipitation (mm),Wind Speed (km/h),Wind Direction (deg),Air Pressure (hPa),condition,City
0,2015-01-01 00:00:00,2015-01-01 00:00:00,6.1,0.6,68.0,NaN,11.2,330.0,1028.6,NaN,Glendale
1,2015-01-01 01:00:00,2015-01-01 01:00:00,5.0,0.6,73.0,0.0,9.4,330.0,1028.9,NaN,Glendale
2,2015-01-01 02:00:00,2015-01-01 02:00:00,4.4,0.0,73.0,0.0,7.6,310.0,1029.0,NaN,Glendale
3,2015-01-01 03:00:00,2015-01-01 03:00:00,4.4,0.0,73.0,0.0,0.0,NaN,1028.5,NaN,Glendale
4,2015-01-01 04:00:00,2015-01-01 04:00:00,3.3,-0.5,76.0,0.0,9.4,320.0,1028.9,NaN,Glendale
5,2015-01-01 05:00:00,2015-01-01 05:00:00,2.8,-1.0,76.0,0.0,11.2,320.0,1028.3,NaN,Glendale
6,2015-01-01 06:00:00,2015-01-01 06:00:00,2.8,-1.7,72.0,0.0,7.6,330.0,1028.0,NaN,Glendale
7,2015-01-01 07:00:00,2015-01-01 07:00:00,2.2,-1.8,75.0,0.0,11.2,320.0,1028.1,NaN,Glendale
8,2015-01-01 08:00:00,2015-01-01 08:00:00,1.7,-2.3,75.0,0.0,9.4,330.0,1028.0,NaN,Glendale
9,2015-01-01 09:00:00,2015-01-01 09:00:00,1.7,-3.4,69.0,0.0,0.0,NaN,1027.4,NaN,Glendale


loop through directory

followed by grabbing 2 names that are similar (i have list somewhere)

make 2 dataframes of them, then append 2 to 1 so it's 2010-2019 complete weather history hourly